In [65]:
import geopandas as gpd
from shapely import difference
import os

In [36]:
brfn_gdb = r'\\spatialfiles.bcgov\work\srm\nr\NEGSS\NEDD\First_Nations_Agreements\BRFN_Implementation_Agreement\FileGeodatabase\BRFNAgreement.gdb'
gdb = r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance\HV1_Disturbance_Analysis.gdb'

hv1 = gpd.read_file(brfn_gdb, layer = 'BRFN_HV1')
hv1_plan = hv1[ hv1['OBJECTID'].isin([19, 23, 29, 36, 4, 32, 18, 2, 27, 35, 3, 9]) ]

w2w = gpd.read_file(gdb, layer='W2W_Dist_Merge')

c:\Users\nross\AppData\Local\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:198: UserWarning: Measured (M) geometry types are not supported. Original type 'Measured 3D MultiPolygon' is converted to 'MultiPolygon Z'
  return ogr_read(


In [32]:
list(w2w.columns)

['CutType',
 'BuffDist',
 'Year',
 'SLUCode',
 'MERGE_SRC',
 'Shape_Length',
 'Shape_Area',
 'geometry']

In [37]:
w2w.loc[ w2w['MERGE_SRC'].str.contains('Well'), 'SLUCode'] = 'WELLSITE_FACILITY'
w2w.loc[ w2w['MERGE_SRC'].str.contains('Pipeline'), 'SLUCode'] = 'PIPELINE'
w2w.loc[ w2w['MERGE_SRC'].str.contains('Road'), 'SLUCode'] = 'ROAD'
w2w.loc[ w2w['MERGE_SRC'].str.contains('Seismic'), 'SLUCode'] = 'GEOPHYSICAL'
w2w.loc[ w2w['MERGE_SRC'].str.contains('Ancillary'), 'SLUCode'] = 'ANCILLARY'


w2w.loc[ w2w['SLUCode'] == 'GEOPHYSICAL', 'SeismicSrc'] = w2w['MERGE_SRC']


In [38]:
w2w['SLUCode'].unique()

array(['WELLSITE_FACILITY', 'PIPELINE', 'ROAD', 'GEOPHYSICAL',
       'ANCILLARY'], dtype=object)

In [81]:
# Note - "CutType" uses the CutType field from Seismic_1996_2004, Seismic_2002_2006, and Seismic_Permitted. There is no field for this in LEGACY2D lines
# LineWidth uses the "LineWidth" field for all sources except Legacy2D, which uses "Width_Code".
# Width Codes are 0, 3, 5, 7. This is the approx lower limit of the width in meters

# Fix discrepancy in caps on MECH lines
w2w.loc[w2w['CutType'] == 'MECH', 'CutType'] = 'Mech'


In [84]:
w2w.to_file(r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance\w2w_cleaned.shp')

C:\Users\nross\AppData\Local\Temp\ipykernel_35548\221098958.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  w2w.to_file(r'\\spatialfiles.bcgov\work\srm\fsj\Workarea\nross\WMBPlanning\Data\Disturbance\w2w_cleaned.shp')
c:\Users\nross\AppData\Local\anaconda3\envs\geo_env\Lib\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'Shape_Length' to 'Shape_Leng'
  ogr_write(


In [77]:
# Dissolve on all fields
w2w.geometry = w2w.geometry.make_valid()

w2w_dis = w2w.dissolve(by=['SLUCode'])

In [78]:
slu_ranked = {
    4: 'WELLSITE_FACILITY',
    3: 'PIPELINE',
    2: 'ROAD',
    1: 'GEOPHYSICAL',
    0: 'ANCILLARY'
}
w2w_erase = w2w_dis

for rank in range(0,5):
    mask = w2w_erase.index == slu_ranked[rank]
    # Get geom of current SLU
    geom = w2w_erase.loc[ mask, 'geometry']
    
    # recursively erase the higher-ranking SLU classes from it
    for upper_rank in range(rank+1, 5):
        geom = difference(geom, w2w_erase.loc[ w2w_erase.index == slu_ranked[upper_rank], 'geometry'])
    
    # write this new geom to output
    w2w_erase.loc[mask, 'geometry'] = geom

In [ ]:
import folium
m = w2w_dis.explore(name='erase')
m = w2w.explore(m=m, color="red", name='w2w')

folium.LayerControl().add_to(m)
m

In [47]:
w2w_dis = w2w_dis[['geometry']]

w2w_hv1 = w2w_dis.overlay(hv1_plan, how='intersection')